# Project 5: Multimodal Models

In this project, you will explore multimodal tasks that interface language with vision, such as image retrieval, caption retrieval, and image captioning. We will work with [Concadia](https://arxiv.org/abs/2104.08376), an image-text dataset scraped from Wikipedia focused on the relationship between images and the textual context. First, you will use CLIP for image retrieval and caption retrieval. Then, we will explore how implementing a Rational Speech Acts inference procedure can improve retrieval performance. Finally, we will generate our own captions for these images.

## Setup

First, we will install the required dependencies and download the dataset.

In Kaggle, you may want to turn on file persistence (Right Sidebar > Session Options > Persistence > Files only) so you don't have to re-download the dataset every time you restart this notebook.

In [ ]:
%%capture
!pip install transformers
!pip install gdown

In [ ]:
%%capture
# Download dataset
# resized.zip
!gdown --id 1gDhVlOwcGcwBT5LWYwgn9xEElGlKVpFb
!unzip /kaggle/working/resized.zip
# wiki_split.json
!gdown --id 1kiTSiqk7y7JdHssXjoLwcOomC7lhb5k8

In [ ]:
import json
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer, AutoProcessor, AutoModelForCausalLM
import torch

Before proceeding, let's verify that we're connected to a GPU runtime and that torch can detect the GPU. We'll define a variable device here to use throughout the code so that we can easily change to run on CPU for debugging.

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print("Using device:", device)

## Dataset Setup

For this project, let's load up the validation split of the dataset and inspect a sample.

Concadia consists of images paired with _captions_, which provide additional context, and _descriptions_, which are intended to replace the image (e.g. for alt-text). We'll be comparing captions and descriptions for various multimodal tasks, so feel free to explore what those instances look like. You may also want to refer to the paper for context.

In [ ]:
with open("/kaggle/working/wiki_split.json") as f:
    data = json.load(f)["images"]
val = [d for d in data if d["split"] == "val"]
print("Number of validation examples:", len(val))

In [ ]:
# Inspect a data sample
img = Image.open(f"/kaggle/working/resized/{val[0]['filename']}")
display(img)
print("Caption:", val[0]["caption"]["raw"])
print("Description:", val[0]["description"]["raw"])

## Part 1: Image & Text Retrieval


First, we'll retrieve images from the dataset from captions and descriptions, and evaluate the relative performance of retrieving with captions vs. descriptions for a subset of the validation set.

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Evaluate on a subset of the validation set
MAX_LEN = 70
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
data = [d for d in val if len(tokenizer(d["description"]["raw"])["input_ids"]) < MAX_LEN
        and len(tokenizer(d["caption"]["raw"])["input_ids"]) < MAX_LEN]
data = data[:1000]

In [ ]:
def rank_images(queries, images):
    """Ranks images for a batch of text queries by their CLIP similarity score.
    Args:
        queries: a list with batch of N text queries
        images: a list with a batch of N images
    Returns:
         A NxN array with image indices sorted by retrieval score for each query.
    """
    # YOUR CODE HERE
    ##

You will implement several retrieval metrics:
* Mean reciprocal rank: the average 1/rank of the true image for a query
* Top-1 accuracy: the % of instances where the true image is the top result for a query
* Top-k accuracy (k=10): the % of instances where the true image is in the top k results for a query

In [ ]:
def mrr(batch_outputs):
    # YOUR CODE HERE
    ###

def top1_accuracy(batch_outputs):
    # YOUR CODE HERE
    ### 
    
def topk_accuracy(batch_outputs, k=10):
    # YOUR CODE HERE
    ###

Now we'll compute each of these metrics for retrieving images based on captions vs. descriptions.

You should be getting >.60 MRR, >50% top-1 accuracy and >80% top-10 accuracy for both.

In [ ]:
captions = [ex["caption"]["raw"] for ex in data]
descs = [ex["description"]["raw"] for ex in data]
images = [Image.open(f"/kaggle/working/resized/{ex['filename']}") for ex in data]

In [ ]:
batch_outputs_cap = rank_images(captions, images)
print("Caption-based retrieval:")
print("MRR: ", mrr(batch_outputs_cap))
print("Top-1 acc: ", top1_accuracy(batch_outputs_cap))
print("Top-10 acc: ", topk_accuracy(batch_outputs_cap))
np.save("image_retrieval_caption.npy", batch_outputs_cap)

batch_outputs_desc = rank_images(descs, images)
print("Descriptions-based retrieval:")
print("MRR: ", mrr(batch_outputs_desc))
print("Top-1 acc: ", top1_accuracy(batch_outputs_desc))
print("Top-10 acc: ", topk_accuracy(batch_outputs_desc))
np.save("image_retrieval_desc.npy", batch_outputs_desc)

Now we'll look at the inverse problem: for a given image, does the corresponding caption or description in the dataset score higher?

In [ ]:
def caption_better(captions, descriptions, images):
    """For a batch of paired (caption, description, image), returns True for each batch element
    if the caption scores higher for the image than the description.
    Args:
        captions: a list with batch of N text captions
        descriptions:  list with batch of N text descriptions
        images: a list with a batch of N images
    Returns:
         A list of N booleans, True if the caption scores higher for the image
    """
    # YOUR CODE HERE
    ##
    
print("% of examples where caption scores more highly:", np.mean(caption_better(captions, descs, images)))

You should see >50% of examples where the caption scores more highly.

### Qualitative Analysis Part 1: Image & Text Retrieval

In the report, provide the following analysis:
* Investigate a couple of examples where image retrieval fails, and systematic model issues you observe (if any).
* Are there any differences between the failure modes of retrieving based on captions and retrieving based on descriptions (e.g. for the same example)? You may also look at some examples of caption and description text and comment on where you think their performance could differ in general.
* Discuss how the retrieval metrics affect analysis. All of these metrics are commonly reported in information retrieval — how do the metrics (and your qualitiative analysis from the last two steps) provide different perspectives on model performance? 
* Investigate a couple of examples where the paired caption scores more highly than the paired description for an image. Are there any patterns you observe that determine whether the caption or the description scores more highly?

In [ ]:
# YOUR CODE HERE
###

## Part 2: Rational Speech Acts (RSA)

Now we will implement RSA on top of our simple image retrieval system. Recall that the idea of RSA is to model how likely an utterance is _in context_. Even if a caption scores the highest for a given image based on raw similarity score, it may no longer be the best text in the context of other, alternative captions for the image. We will implement this idea and investigate how it changes retrieval performance.

We'll work with the examples from Part 1 where the true image is not in the top 10 retrieval results for a given caption. You should double-check your retrieval performance matches the expected numbers before you proceed; otherwise, you'll be working with different examples here. 

In [ ]:
incorrect = [b[:10] for i, b in enumerate(batch_outputs_cap) if i not in b[:10]]
incorrect_idxs = [i for i, b in enumerate(batch_outputs_cap) if i not in b[:10]]
assert len(incorrect) == 92, "Your retrieval system results do not match ours."

For each of the captions with an incorrect retrieval result, we'll get the top 10 retrieved images and the true captions for each of those 10 images. The alternative captions will form the context for RSA.

Let's implement the RSA functions that we'll run on these contexts.

First, implement a **literal listener** which retrieves the top images for a given caption without modeling how likely a speaker is to generate those captions.
    
This should look similar to your rank_images retrieval function from Part 1, except the function should now return softmax-normalized probabilities (which will then be used by the pragmatic speaker).

In [ ]:
def literal_listener(captions, images):
    """
    Args:
        captions: list of N captions
        images: list of N images
    Returns:
        a NxN array of normalized conditional probabilities,
        where A[i][j] = p(image j | caption i)
    """
    # YOUR CODE HERE
    ###
    assert np.allclose(listener_probs.sum(1), 1)
    return listener_probs

Now, we'll implement a **pragmatic speaker**. For a given image, the pragmatic speaker is more likely to select utterances that elicit the right inference in the listener (i.e. the more likely the listener is to choose the right image based on the caption, the more likely the speaker is to use that caption).

The pragmatic speaker should call the literal_listener within the function. Normalize your speaker probabilities by dividing by the sum (rather than using softmax), matching the implementation in lecture.

In [ ]:
def pragmatic_speaker(captions, images):
    """
    Args:
        captions: list of N captions
        images: list of N images
    Returns:
        a NxN array of normalized conditional probabilities,
        where A[i][j] = p(caption i | image j)
    """
    # YOUR CODE HERE
    ###
    assert np.allclose(speaker_probs.sum(0), 1)
    return speaker_probs

Finally, implement a **pragmatic listener**, which models how a pragmatic speaker would select utterances. Similarly, you should call pragmatic_speaker within the function and use sum-normalization instead of softmax.

In [ ]:
def pragmatic_listener(captions, images):
    """
    Args:
        captions: list of N captions
        images: list of N images
    Returns:
        a NxN array of normalized conditional probabilities,
        where A[i][j] = p(image j | caption i)
    """
    # YOUR CODE HERE
    ###
    assert np.allclose(listener_probs.sum(1), 1)
    return listener_probs

Now let's evaluate the pragmatic listener on each context. You should get >20 examples (out of 92) where pragmatics fixes the error.

In [ ]:
improved = 0
predictions = []

for true_idx, topk_idxs in zip(incorrect_idxs, incorrect):
    img_context = [images[idx] for idx in [true_idx] + topk_idxs.tolist()]
    caption_context = [captions[idx] for idx in [true_idx] + topk_idxs.tolist()]
    # Call your pragmatic listener and increment `improved` if the top image candidate is now correct
    # YOUR CODE HERE
    ###
    # Save prediction outputs
    predictions.append(pragmatic_probs)
        
print(f"# examples where pragmatics fixes the error: {improved} / {len(incorrect)}")
np.save("pragmatic_preds.npy", np.array(predictions))

## Qualitative Analysis Part 2: RSA

In your report:
* Report the # of examples that were previously incorrect that are now correct (the correct image is now the top candidate).
* Investigate some qualitative examples to get an intuition of why RSA fixes these issues and discuss one of these examples (e.g. compare the literal listener probabilities to the pragmatic listener probabilities and think about why they changed). 

In [ ]:
# YOUR CODE HERE
###

## Part 3: Image Captioning

Finally, we'll explore caption generation using a pretrained model, and how RSA can help us generate more discriminative captions.

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

cap_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
cap_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16).to("cuda")

In [ ]:
# Generate image captions for a subset of data
people = [d for d in val if " man " in d["description"]["raw"] or " woman " in d["description"]["raw"]]
people = people[:100]
people_images = [Image.open(f"/kaggle/working/resized/{ex['filename']}") for ex in people]

We'll compare the image retrieval performance for a literal captioner to that of a pragmatic captioner.

First, implement a literal captioner by just decoding the greedy output from the base captioning model.

In [ ]:
def literal_captioner(image):
    # YOUR CODE HERE
    ###

Now, implement a pragmatic captioner by sampling candidate captions from a base captioning model, and then using the `pragmatic_speaker` function you implemented earlier to re-rank captions using the other images in the subset of 100 images of people to normalize your speaker probabilities. Use temperature 0.6 and top_p 0.95 for sampling.

In [ ]:
def generate_candidate_captions(image, k=10):
    """Generate k candidate captions for an image."""
    # YOUR CODE HERE
    ###
    assert isinstance(candidates, list) and len(candidates) == k and isinstance(candidates[0], str)
    return candidates
    
def pragmatic_captioner(image, context_images):
    """Generate a caption for the image using a set of context images."""
    # Generate candidate captions
    # YOUR CODE HERE
    ###
    # Rank captions with the pragmatic_speaker and return the best one
    pragmatic_caption = ... # YOUR CODE HERE
    return pragmatic_caption

In [ ]:
results = []

for i in range(len(people_images)):
    literal_caption = literal_captioner(people_images[i])
    # Call the pragmatic captioner and return the caption after RSA and the candidate captions
    # Be careful when constructing the context images, which should not include the true image.
    pragmatic_caption = ... # YOUR CODE HERE
    results.append((literal_caption, pragmatic_caption))

In [ ]:
# Inspect some results
for img, r in zip(people_images[:10], results[:10]):
    display(img)
    print("Literal caption: ", r[0])
    print("Pragmatic caption: ", r[1])

Now, if we were to do image retrieval with our generated captions, let's take a look at how retrieval performance changes if we retrieve based on the greedy output from the captioner vs. the more discriminative captions selected by RSA.

Your top-1 retrieval accuracy with pragmatic captions should be >70%.

In [ ]:
literal_retrieval_outputs = rank_images([r[0] for r in results], people_images)
print("Top-1 accuracy (literal):", top1_accuracy(literal_retrieval_outputs))
pragmatic_retrieval_outputs = rank_images([r[1] for r in results], people_images)
print("Top-1 accuracy (pragmatic):", top1_accuracy(pragmatic_retrieval_outputs))

np.save("image_retrieval_pragmatic.npy", pragmatic_retrieval_outputs)

## Qualitative Analysis Part 3: Captioning

In your report:
* Look at a few examples of captions generated by the literal and the pragmatic captioner. What are some qualitative trends and why would they come out of RSA inference?

# Submission

Your final submission should include the following files:

* hw5.ipynb (this file; please rename to match)
* image_retrieval_caption.npy
* image_retrieval_desc.npy
* pragmatic_preds.npy
* image_retrieval_pragmatic.npy
* report.pdf (containing qualitative analysis for Parts 1-3)